In [1]:
df = spark.read.format('csv').option('header','true').option('inferSchema','true').load('/FileStore/tables/retail_data.csv')
df.show()

In [2]:
from pyspark.sql.functions import window, column, desc, col
df.selectExpr("customerId" ,"(UnitPrice * Quantity) as total_cost","InvoiceDate").groupBy("customerId", window(col("InvoiceDate"),"1 day") ).sum("total_cost").show()

In [3]:
reatilData_schema = df.schema
reatilData_schema

In [4]:
stream_df = spark.readStream.schema(reatilData_schema)\
.option("maxFilesPerTrigger",1)\
.format("csv")\
.option("header","true")\
.load('/FileStore/tables/retail_data.csv')

stream_df.isStreaming
#stream_df.show()

In [5]:
# still a lazy operation
from pyspark.sql.functions import window, column, desc, col
df_perHour = stream_df.selectExpr("customerId" ,"(UnitPrice * Quantity) as total_cost","InvoiceDate").groupBy("customerId", window(col("InvoiceDate"),"1 day") ).sum("total_cost")

In [6]:
spark.conf.set("spark.sql.shuffel.partitions","5")

In [7]:
df_perHour.writeStream.format("memory")\
.queryName("abc")\
.outputMode("complete")\
.start()
#http://spark.apache.org/docs/latest/streaming-programming-guide.html#checkpointing

In [8]:
spark.sql("select * from abc").show()

In [9]:
df_perHour.writeStream.format("console")\
.queryName("abc")\
.outputMode("complete")\
.start()